In [119]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold,train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import xgboost as xgb


In [120]:
data = pd.read_csv(r"C:\Users\aloka\Downloads\Train (1).csv")

In [121]:
data.head()

,Inv_Id,Vendor_Code,GL_Code,Inv_Amt,Item_Description,Product_Category
0,15001,VENDOR-1676,GL-6100410,83.24,Artworking/Typesetting Production Jun 2009 Cha...,CLASS-1963
1,15002,VENDOR-1883,GL-2182000,51.18,Auto Leasing Corporate Services Corning Inc /N...,CLASS-1250
2,15004,VENDOR-1999,GL-6050100,79.02,Store Management Lease/Rent Deltona Corp Real ...,CLASS-1274
3,15005,VENDOR-1771,GL-6101400,48.50,Store Construction General Requirements Coloni...,CLASS-1522
4,15006,VENDOR-1331,GL-2182000,63.35,Jul 2015 Aydin Corp Contingent Labor/Temp Labo...,CLASS-1376


In [122]:
data.shape

(5566, 6)

In [123]:
data.columns

Index(['Inv_Id', 'Vendor_Code', 'GL_Code', 'Inv_Amt', 'Item_Description',
       'Product_Category'],
      dtype='object')

In [124]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5566 entries, 0 to 5565
Data columns (total 6 columns):
Inv_Id              5566 non-null int64
Vendor_Code         5566 non-null object
GL_Code             5566 non-null object
Inv_Amt             5566 non-null float64
Item_Description    5566 non-null object
Product_Category    5566 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 261.0+ KB


In [125]:
len(data['Product_Category'].unique())

36

In [126]:

data['Product_Category'].value_counts().sort_values(ascending=False)

CLASS-1758    1498
CLASS-1274     986
CLASS-1522     851
CLASS-1250     451
CLASS-1376     365
CLASS-1963     230
CLASS-1249     176
CLASS-1828     113
CLASS-2141     108
CLASS-1721     107
CLASS-1567      84
CLASS-1919      70
CLASS-1850      60
CLASS-2112      53
CLASS-1477      50
CLASS-1870      36
CLASS-2241      36
CLASS-1309      32
CLASS-2003      32
CLASS-1429      32
CLASS-1964      28
CLASS-1322      28
CLASS-1294      25
CLASS-1770      19
CLASS-1983      17
CLASS-1652      15
CLASS-1867      15
CLASS-2038      13
CLASS-1805      12
CLASS-2152       9
CLASS-1248       5
CLASS-1688       4
CLASS-2146       2
CLASS-2015       2
CLASS-1957       1
CLASS-1838       1
Name: Product_Category, dtype: int64

In [127]:
#encode string to numerical
from sklearn.preprocessing import LabelEncoder
en = LabelEncoder()
for col in ['GL_Code', 'Vendor_Code','Product_Category']:
    
    data[col] = en.fit_transform(data[col])
    
   


In [128]:
#seperate Product_Category
y = data['Product_Category']
data.drop(['Product_Category'],axis=1,inplace=True)


In [117]:
# Text Cleaning
remove_words = set(stopwords.words('english'))

def token(text):
     
    sent =re.sub("[^A-Za-z]+"," ",text)
    sents = sent.split()
    
    clean_tokens = []
    for x in sents:
        if x not in string.punctuation and x not in stop_words:
            clean_tokens.append(x.lower().strip())
    
    return clean_tokens
        

In [129]:
#Apply TfidfVectorizer

model_vect = TfidfVectorizer(tokenizer=token, ngram_range=(1,2), use_idf = False, max_features = None)
model_vect.fit(data['Item_Description'])

X = model_vect.transform(data['Item_Description'])

In [130]:
layer = {}
# use softmax multi-class classification
layer['objective']= 'multi:softprob'
layer['eta'] = 0.1
layer['max_depth'] = 6
layer['silent']=1
layer['nthread'] = 4
layer['num_class'] = len(en.classes_)
layer['eval_metric'] = ['mlogloss']
layer['seed'] = 1


In [132]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=1)

m_train = xgb.DMatrix(X_train, label=y_train)
m_test = xgb.DMatrix(X_valid, label=y_valid)

evaluation = [(m_train, 'train'), (m_test, 'eval')]

model = xgb.train(layer, dtrain, 100, evaluation, verbose_eval=50)

[0]	train-mlogloss:2.02698	eval-mlogloss:2.02699
[50]	train-mlogloss:0.014214	eval-mlogloss:0.020119
[99]	train-mlogloss:0.003764	eval-mlogloss:0.010158


In [89]:
y_pred_valid = model.predict(dvalid)

print("Accuracy : ",accuracy_score(y_valid, np.argmax(y_pred_valid, axis=1)))

Accuracy :  0.9988023952095808
